In [ ]:
#Name : Arnob Bokshi
#ID : 2125051010
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
movie_df_raw = pd.read_csv("/content/drive/MyDrive/Dataset/Recommendation System /ratings.csv")
movie_names = pd.read_csv("/content/drive/MyDrive/Dataset/Recommendation System /movies.csv")

print(movie_df_raw.head())
print(movie_names.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [ ]:
movie_data = movie_df_raw.merge(movie_names, on="movieId")

# Display first few rows
print(movie_data.head())


   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


In [ ]:
user_movie_matrix = movie_data.pivot_table(index="userId", columns="title", values="rating")
user_movie_matrix.fillna(0, inplace=True)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarity between movies
movie_similarity = cosine_similarity(user_movie_matrix.T)

# Convert into a DataFrame
movie_similarity_df = pd.DataFrame(movie_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)


In [ ]:
def recommend_similar_movies(movie_name, top_n=5):
    similar_scores = movie_similarity_df[movie_name].sort_values(ascending=False)
    return similar_scores.iloc[1:top_n+1]

# Example: Find movies similar to "Toy Story (1995)"
print(recommend_similar_movies("Toy Story (1995)"))


title
Toy Story 2 (1999)                           0.572601
Jurassic Park (1993)                         0.565637
Independence Day (a.k.a. ID4) (1996)         0.564262
Star Wars: Episode IV - A New Hope (1977)    0.557388
Forrest Gump (1994)                          0.547096
Name: Toy Story (1995), dtype: float64


In [ ]:
def recommend_for_user(user_id, top_n=5):
    # Get movies rated highly by the user
    user_ratings = movie_data[movie_data["userId"] == user_id].sort_values(by="rating", ascending=False)
    top_movies = user_ratings["title"].head(5)  # Take top-rated movies

    recommendations = []
    for movie in top_movies:
        similar_movies = recommend_similar_movies(movie, top_n=2)
        recommendations.extend(similar_movies.index.tolist())

    return set(recommendations)

# Example: Recommend for User 1
print(recommend_for_user(1))


{'Grifters, The (1990)', 'Live and Let Die (1973)', 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Lord of the Rings, The (1978)', 'Indiana Jones and the Temple of Doom (1984)', 'Black Cauldron, The (1985)', 'Butch Cassidy and the Sundance Kid (1969)', 'Stand and Deliver (1988)', 'Thunderball (1965)', 'Valmont (1989)'}
